# Weekly project

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [ ]:
import numpy as np
import cv2 as cv2
from numpy.linalg import inv, pinv
import matplotlib.pyplot as plt
import time as t
from helpers import *

def extract_keypoints_surf(img1, img2, K, baseline):
    """
    use surf to detect keypoint features
    remember to include a Lowes ratio test
    """
    surf = cv2.SIFT_create()
    # Find keypoints and descriptors directly
    kp1, des1 = surf.detectAndCompute(img1,None)
    kp2, des2 = surf.detectAndCompute(img2,None)   
    
    bf = cv2.BFMatcher()
    
    matches = bf.knnMatch(des1, des2, k=2)
    #print(matches)
    #good = []
    #for i, (m, n) in enumerate(matches):
    #    if m.distance < 0.7*n.distance:
    #        good.append(m)
            
    # ratio test as per Lowe's paper
    match_points1, match_points2 = [], []
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            match_points1.append(kp1[m.queryIdx].pt)
            match_points2.append(kp2[m.trainIdx].pt)
            
    #print([(m.trainIdx, m.queryIdx) for m in good])
    #match_points1 = [kp1[m.queryIdx].pt for m in good]
    #match_points2 = [kp2[m.trainIdx].pt for m in good]
    
    #print(len(match_points1), len(match_points2))
    
    p1 = np.array(match_points1).astype(np.float32)
    p2 = np.array(match_points2).astype(np.float32)

    ##### ############# ##########
    ##### Do Triangulation #######
    ##### ########################
    #project the feature points to 3D with triangulation
    
    #projection matrix for Left and Right Image
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))

    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1

def featureTracking(img_1, img_2, p1, world_points):
    """
    fill in
    """
    p2, status, _ = cv2.calcOpticalFlowPyrLK(img_1, img_2, p1, None, None, None, winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    #print(next_points)
    
    st = status.reshape(status.shape[0])
    ##find good one
    pre = p1[st==1]
    p2 = p2[st==1]
    w_points  = world_points[st==1]
    
    #p1 = p1[np.where(np.ravel(status) == 1)]
    #p2 = p2[np.where(np.ravel(status) == 1)] 
    #world_points = world_points[np.where(np.ravel(status) == 1)]

    return w_points, pre, p2

def playImageSequence(left_img, right_img, K):

    baseline = 0.54;

    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################
    """
    Implement step 1.2 and 1.3
    Store the features in 'reference_2D' and the 3D points (landmarks) in 'landmark_3D'
    hint: use 'extract_keypoints_surf' above
    """
    landmark_3D, reference_2D = extract_keypoints_surf(left_img, right_img, K, baseline)
    #landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(left_img, 
    #                                                              right_img, 
    #                                                              reference_2D,
    #                                                              landmark_3D)

    # reference
    reference_img = left_img

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8);
    maxError = 0

    for i in range(0, 1400):
        print('image: ', i)
        curImage = getLeftImage(i)
        curImage_R = getRightImage(i)

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        #landmark_3D, reference_2D = extract_keypoints_surf(reference_img, curImage, K, baseline)
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, 
                                                                      curImage, 
                                                                      reference_2D,
                                                                      landmark_3D)

        ##### ################################# #######
        ##### Calculate relative pose using PNP #######
        ##### #########################################
        """
        Implement step 2.3)
        """
        #print(type(landmark_3D), type(reference_2D))
        #pnp_objP = np.expand_dims(landmark_3D, axis = 2)
        #pnp_cur  = np.expand_dims(tracked_2Dpoints, axis = 2).astype(float)

        #_, rvec, tvec, inliers = cv2.solvePnPRansac(pnp_objP , pnp_cur, K, None)
        _, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, None)

        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)
        tvec = -rot.T.dot(tvec)  # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        inv_transform = np.hstack((rot.T, tvec))  # inverse transform. A tranform projecting points from the camera frame to the world frame

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points
        """
        Implement step 2.4)
        """
        landmark_3D_new, reference_2D_new = extract_keypoints_surf(curImage, curImage_R, K, baseline)
        
        #Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')
        landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0])))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100);
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]));
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2);
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2);
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED);
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8);

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis);
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

if __name__ == '__main__':
    left_img = getLeftImage(0)
    right_img = getRightImage(0)

    K = getK()

    playImageSequence(left_img, right_img, K)

image:  0
[-0.00076] [-0.00077691] [-0.00040673] [-8.34502269e-05] [-5.48880132e-05] [5.67321315e-05]
[5.551115e-17, 3.330669e-16, -4.440892e-16]
image:  1
[-0.00235542] [-0.00660061] [0.67375536] [-0.00233391] [0.00330903] [-0.00248072]
[-0.04690294, -0.02839928, 0.8586941]
image:  2
[-0.01549488] [-0.01285307] [1.37050334] [-0.00400179] [0.00730648] [-0.00079479]
[-0.09374345, -0.05676064, 1.716275]
image:  3
[-0.0314307] [-0.02080658] [2.08179928] [-0.00546712] [0.01120414] [-0.00067545]
[-0.1406429, -0.08515762, 2.574964]
image:  4
[-0.05220452] [-0.02841354] [2.81381614] [-0.00620376] [0.0160319] [0.00084305]
[-0.1874858, -0.1135202, 3.432648]
image:  5
[-0.06852236] [-0.04087186] [3.55649518] [-0.00637893] [0.02058984] [-0.00014946]
[-0.2343818, -0.141915, 4.291335]
image:  6
[-0.09180883] [-0.0532135] [4.31544939] [-0.00769676] [0.02527201] [0.00192076]
[-0.2812195, -0.1702743, 5.148987]
image:  7
[-0.12481079] [-0.06552596] [5.09071667] [-0.00832593] [0.0301261] [0.004656]
[-0.

[-3.86403175] [-0.85164502] [59.48435215] [-0.00474224] [0.07507354] [0.00702097]
[-3.641784, -2.021539, 61.09125]
image:  66
[-3.942116] [-0.87352645] [60.38424047] [-0.00973475] [0.07575051] [0.009616]
[-3.703384, -2.049159, 62.00632]
image:  67
[-4.02662292] [-0.8976358] [61.29124722] [-0.00929268] [0.07659859] [0.01546716]
[-3.776035, -2.078109, 62.91678]
image:  68
[-4.10752323] [-0.92607525] [62.17896438] [-0.00712634] [0.07735828] [0.01882235]
[-3.84394, -2.112094, 63.81519]
image:  69
[-4.19138067] [-0.95098954] [63.06875535] [-0.00589978] [0.07811926] [0.02187071]
[-3.912602, -2.145353, 64.69786]
image:  70
[-4.26990987] [-0.97409687] [63.93325587] [-0.00633955] [0.07906848] [0.02318843]
[-3.978228, -2.173835, 65.56672]
image:  71
[-4.34295505] [-0.99823302] [64.78103949] [-0.00556995] [0.08070169] [0.02524172]
[-4.046625, -2.201409, 66.42163]
image:  72
[-4.41984926] [-1.02172721] [65.62117405] [-0.00496948] [0.08198296] [0.02636063]
[-4.114442, -2.230822, 67.26455]
image:  7

[4.90325687] [-2.1750301] [88.28324621] [-0.07500303] [-1.50990756] [0.02475115]
[5.460049, -3.327578, 89.60332]
image:  132
[5.42279738] [-2.20530547] [88.29984164] [-0.07873274] [-1.5199697] [0.02292313]
[5.974572, -3.344693, 89.62101]
image:  133
[5.9543841] [-2.24112226] [88.31630076] [-0.08147551] [-1.52776305] [0.02136555]
[6.504623, -3.36274, 89.63826]
image:  134
[6.50239154] [-2.27825638] [88.32871324] [-0.08376088] [-1.5331812] [0.01998209]
[7.050381, -3.379767, 89.65473]
image:  135
[7.0641838] [-2.31829664] [88.34054994] [-0.08615543] [-1.53863067] [0.01951276]
[7.614093, -3.394968, 89.67153]
image:  136
[7.64444022] [-2.35841578] [88.34969934] [-0.08785339] [-1.54448084] [0.0190766]
[8.197957, -3.409306, 89.68785]
image:  137
[8.246026] [-2.40349159] [88.35625163] [-0.09023818] [-1.54871424] [0.0169835]
[8.800912, -3.4262, 89.70167]
image:  138
[8.87052856] [-2.44946125] [88.36279658] [-0.09187654] [-1.55151689] [0.01560708]
[9.42023, -3.442284, 89.70786]
image:  139
[9.51

[51.46329645] [-6.12393895] [87.98623743] [-0.06500444] [-1.45909349] [0.05723675]
[51.96384, -5.139956, 89.33556]
image:  198
[51.96138879] [-6.16744994] [88.09060924] [-0.0593048] [-1.41313176] [0.05639174]
[52.46407, -5.168307, 89.45093]
image:  199
[52.44970804] [-6.20433129] [88.21897585] [-0.05277411] [-1.36335433] [0.05802162]
[52.95984, -5.197886, 89.59271]
image:  200
[52.92231316] [-6.24272026] [88.37798824] [-0.04719435] [-1.30986867] [0.06048366]
[53.43056, -5.228395, 89.75037]
image:  201
[53.38478715] [-6.26591216] [88.55641988] [-0.04676567] [-1.25137024] [0.0620637]
[53.88945, -5.254058, 89.94526]
image:  202
[53.82999325] [-6.30164213] [88.75872794] [-0.04822073] [-1.18825631] [0.06202812]
[54.33154, -5.281038, 90.16419]
image:  203
[54.25333925] [-6.33533987] [88.99833534] [-0.05209905] [-1.12177905] [0.06430373]
[54.73889, -5.307765, 90.3886]
image:  204
[54.6530898] [-6.37866262] [89.25602257] [-0.05558853] [-1.05280691] [0.06577271]
[55.13319, -5.33687, 90.66097]
i

[64.84467525] [-7.68448733] [125.89086311] [-0.01029549] [-0.24376299] [0.1322025]
[65.45904, -6.878751, 127.2746]
image:  263
[65.03222333] [-7.71944378] [126.7243643] [-0.00565766] [-0.24529537] [0.13709852]
[65.65809, -6.913045, 128.1205]
image:  264
[65.22803816] [-7.75409132] [127.55546916] [-0.01261678] [-0.24640425] [0.13640358]
[65.86477, -6.955017, 128.941]
image:  265
[65.43063866] [-7.79256498] [128.36543238] [-0.02810419] [-0.24568479] [0.13102676]
[66.08066, -7.000709, 129.7485]
image:  266
[65.63874122] [-7.83206999] [129.18008649] [-0.03800356] [-0.24444987] [0.12822288]
[66.29836, -7.046839, 130.5573]
image:  267
[65.8343681] [-7.86658829] [130.00575244] [-0.02943593] [-0.24435557] [0.12686813]
[66.51051, -7.092137, 131.3828]
image:  268
[66.03558277] [-7.90603531] [130.84538836] [-0.00921838] [-0.2442423] [0.12278586]
[66.72265, -7.134533, 132.2254]
image:  269
[66.24044639] [-7.9486169] [131.6666566] [0.00581108] [-0.24376656] [0.12108259]
[66.93208, -7.177765, 133.06

...

[-176.35443516] [30.23599487] [232.16864367] [-0.03318905] [-2.98641653] [-0.33438706]
[-179.4618, -0.4651856, 233.8196]
image:  1100
[-176.25484256] [30.39664201] [231.40982003] [-0.03490815] [-2.98897892] [-0.33470377]
[-179.4435, -0.4403554, 233.0489]
image:  1101
[-176.15745754] [30.5525236] [230.66662422] [-0.03566531] [-2.9916458] [-0.33593024]
[-179.4262, -0.4184439, 232.2935]
image:  1102
[-176.06284656] [30.70276187] [229.94194982] [-0.03343018] [-2.99409621] [-0.33663148]
[-179.41, -0.396176, 231.5589]
image:  1103
[-175.97707304] [30.85305677] [229.23469882] [-0.03252115] [-2.99685325] [-0.33679531]
[-179.3952, -0.3719122, 230.8454]
image:  1104
[-175.89316444] [30.99692123] [228.5572145] [-0.03220935] [-2.99897387] [-0.33720398]
[-179.3848, -0.3491587, 230.153]
image:  1105
[-175.8169098] [31.13600589] [227.89603274] [-0.03570087] [-3.0005114] [-0.33780682]
[-179.3801, -0.3282832, 229.4817]
image:  1106
[-175.72973828] [31.27519485] [227.26219392] [-0.03828886] [-3.00206253

[-150.70115747] [32.91613431] [217.13255964] [-0.18048615] [-1.43214395] [-0.12995768]
[-155.4939, -0.3999312, 215.4766]
image:  1162
[-149.86210049] [32.86835046] [217.2550923] [-0.17983016] [-1.43095683] [-0.12588448]
[-154.6509, -0.4260836, 215.4909]
image:  1163
[-149.01112896] [32.81563873] [217.38386946] [-0.18004822] [-1.430586] [-0.12450385]
[-153.798, -0.4491774, 215.5215]
image:  1164
[-148.14995516] [32.76738547] [217.50549747] [-0.1805028] [-1.4288647] [-0.12515314]
[-152.9246, -0.4765393, 215.5288]
image:  1165
[-147.27258026] [32.72126015] [217.6292131] [-0.17936989] [-1.42762547] [-0.13090417]
[-152.043, -0.5017483, 215.5373]
image:  1166
[-146.39106688] [32.67358259] [217.75856319] [-0.17700336] [-1.42633728] [-0.13444805]
[-151.1495, -0.5295159, 215.545]
image:  1167
[-145.50914648] [32.62519944] [217.88635682] [-0.17770717] [-1.42437521] [-0.13505463]
[-150.2503, -0.5601175, 215.5562]
image:  1168
[-144.62429216] [32.58242526] [218.00669363] [-0.18401148] [-1.42092726

[-104.47385755] [29.70801203] [225.07922998] [-0.19805356] [-1.25892271] [-0.0985938]
[-108.8327, -1.995466, 218.1458]
image:  1224
[-103.94712874] [29.64790268] [225.28352139] [-0.19601462] [-1.22167851] [-0.09435893]
[-108.2909, -2.004646, 218.3013]
image:  1225
[-103.43787105] [29.57811591] [225.50763877] [-0.18934747] [-1.18096741] [-0.08897117]
[-107.764, -2.019178, 218.4985]
image:  1226
[-102.93481185] [29.50720113] [225.75911326] [-0.18398246] [-1.13618896] [-0.08441829]
[-107.2563, -2.021587, 218.7282]
image:  1227
[-102.45664309] [29.43607642] [226.02673081] [-0.18492444] [-1.088651] [-0.07955637]
[-106.7495, -2.025929, 218.9454]
image:  1228
[-102.00199857] [29.35961448] [226.31595605] [-0.1857363] [-1.03944982] [-0.07458297]
[-106.2729, -2.029648, 219.2153]
image:  1229
[-101.56863758] [29.27765304] [226.62490629] [-0.18700114] [-0.98730534] [-0.06974521]
[-105.7926, -2.038583, 219.4762]
image:  1230
[-101.14486043] [29.19832184] [226.96247831] [-0.18965145] [-0.93339456] [

[-88.16094048] [24.44169598] [246.21532364] [-0.21351673] [-1.35915498] [-0.07164037]
[-90.60052, -2.985924, 238.2071]
image:  1287
[-87.63008967] [24.38244227] [246.30373242] [-0.21527439] [-1.37033877] [-0.07615124]
[-90.07469, -3.000677, 238.2483]
image:  1288
[-87.07780105] [24.32148249] [246.39600035] [-0.21615722] [-1.37754532] [-0.08030207]
[-89.53204, -3.015435, 238.2882]
image:  1289
[-86.51184505] [24.26232258] [246.49435626] [-0.21590573] [-1.38055726] [-0.08380641]
[-88.96618, -3.031384, 238.3336]
image:  1290
[-85.93048087] [24.19844093] [246.59833074] [-0.21343905] [-1.38099397] [-0.08559343]
[-88.38745, -3.040495, 238.3917]
image:  1291
[-85.33011573] [24.12910105] [246.70567541] [-0.20998831] [-1.38169682] [-0.08663992]
[-87.76457, -3.050019, 238.447]
image:  1292
[-84.70931495] [24.06143184] [246.81959173] [-0.20852989] [-1.38398116] [-0.08605447]
[-87.13484, -3.068087, 238.488]
image:  1293
[-84.07734633] [23.99237914] [246.93323219] [-0.20758805] [-1.38580785] [-0.08

[-39.0098989] [18.69544135] [253.89647996] [-0.21560841] [-1.38207922] [-0.07387916]
[-40.37476, -4.751432, 241.236]
image:  1350
[-38.28658732] [18.59918819] [254.03760415] [-0.21328307] [-1.38173702] [-0.0711887]
[-39.64902, -4.769963, 241.3128]
image:  1351
[-37.58292119] [18.5036577] [254.17222131] [-0.21133416] [-1.38117814] [-0.06787121]
[-38.93985, -4.793176, 241.3846]
image:  1352
[-36.89246806] [18.40858094] [254.30301743] [-0.20960043] [-1.38079076] [-0.06392787]
[-38.24412, -4.801646, 241.4615]
image:  1353
[-36.20453233] [18.31425622] [254.43437242] [-0.20818964] [-1.38123117] [-0.06010659]
[-37.55243, -4.809079, 241.5394]
image:  1354
[-35.52321503] [18.22152829] [254.55996691] [-0.20696138] [-1.38199762] [-0.05707295]
[-36.8629, -4.824751, 241.6097]
image:  1355
[-34.84307699] [18.13000878] [254.68392725] [-0.2061116] [-1.3827796] [-0.05560162]
[-36.17526, -4.839722, 241.6749]
image:  1356
[-34.16387683] [18.04067411] [254.81341483] [-0.20353486] [-1.38391117] [-0.0525350

# Challenge 
The current implementation only uses features computed at the current timestep. However, as we process more images we potentially have a lot of features from previous timesteps that are still valid. The challenge is to expand the `extract_keypoints_surf(..., refPoints)` function by giving it old reference points. You should then combine your freshly computed features with the old features and remove all duplicates. This requires you to keep track of old features and 3D points.

Hint 1: look in `helpers.py` for removing duplicates.

Hint 2: you are not interested in points that are behind you, so remember to remove points that are negative in the direction you move.